In [5]:
    numNodos=4
    rutas=[]
    rutaAux=[]
    alpha=10
    u= 100
    m= 40
    Q= 50
    respuesta=alpha+1.0
    for i in range(numNodos):
        rutaAux.append(1)
    rutas.append(rutaAux)    
    iterador=0
    rutas=[(1,1,0,0),(1,0,0,1),(0,0,1,1),(1,0,1,0),(0,1,0,1),(0,1,1,0),(1,1,1,1),(1,1,1,0),(1,1,0,1),(1,0,1,1),(0,1,1,1)]
    
    numRutas=len(rutas)
    
    #contruir Bod
    b=[]
    aux=[]
    for i in range(numNodos):
         aux.append(0)         
    for i in range(numNodos):
         b.append(aux[:])  
    for i in range(numNodos):
        for j in range(numNodos):
            if(i != j):
                b[i][j]=50
    #contruir Cij                       
    c=[]
    aux=[]
    for i in range(numNodos):
         aux.append(0) 
    for i in range(numNodos):
         c.append(aux[:])  
    for i in range(numNodos):
        for j in range(numNodos):
            if(i != j):
                c[i][j]=abs(i-j)*10-abs(i-j)+1 
    # construir Bodi  

    B2={(o,d,i):0 for o in range(numNodos) for d in range(numNodos) for i in  range(numNodos)}
    for i in range(numNodos):
        for j in range(numNodos):
            B2[i,j,i]=b[i][j]
            B2[i,j,j]=-b[i][j]


    arcos1=[(k,i,j)for k in range(numRutas) for i in range(numNodos) for j in range(numNodos) ]

    #---------------------
    

    numRutas=len(rutas)
    rutasmodificadas=[]
    rutaaux=[]
    for i in range(numRutas):
       rutaaux.clear() 
       for j in range(numNodos):
            if rutas[i][j] == 1:
                rutaaux.append(j)          
       rutasmodificadas.append(rutaaux[:])  
    r={}
    for i in arcos1:
        r[i]=0
    for i in range(numRutas):
        for j in range(len(rutasmodificadas[i])-1):
            r[(i,rutasmodificadas[i][j],rutasmodificadas[i][j+1])]=1
            r[(i,rutasmodificadas[i][j+1],rutasmodificadas[i][j])]=1
    #-------------------------

    arcosX=[(i,j,o,d) for i in range(numNodos)for j in range(numNodos)for o in range(numNodos)for d in range(numNodos)]
    arcosY=[(i) for i in range (numRutas)]

    

    from docplex.mp.model import Model


    mdl= Model("BRT")
    #creacion variables de desicion
    x=mdl.continuous_var_dict(arcosX,name="x")
    y=mdl.continuous_var_dict(arcosY,name="y")


     # funcion a minimizar
    costoX=mdl.sum(c[i][j]*x[(i,j,o,d)]for i in range(numNodos) for j in range(numNodos)for o in range(numNodos)for d in range(numNodos))   
    costoY=mdl.sum((alpha*y[i]) for i in range(numRutas))
    obj=mdl.minimize(costoX+costoY)



    #capacidad  
    capacidad=[]
    for i in  range(numNodos):
        for j in range(numNodos): 
            capacidad.append(mdl.add_constraint(mdl.sum(x[(i,j,o,d)] for o in range(numNodos)for d in range(numNodos))<= mdl.sum(r[(k,i,j)]*y[(k)]*u for k in range(numRutas))))

    #flujo
    flujo=[]
    for i in range(numNodos):  
        for o in range(numNodos):
            for d in range(numNodos):       
                flujo.append(mdl.add_constraint((mdl.sum(x[(i,j,o,d) ] for j in range(numNodos) )-mdl.sum(x[(j,i,o,d) ] for j in range(numNodos) ) )==B2[(o,d,i)]))
    #maximo rutas

    maxRutas=mdl.add_constraint( mdl.sum(y[(k)]for k in range(numRutas))<=m)  


    mdl.solve(log_output=False)
    matriz=[(i,j)for i in range(numNodos) for j in range(numNodos) ]
    vector=[(i) for i in range(numNodos)]
    r={}
    for i in matriz:
        r[i]=0
    for i in range(numRutas):
        if(y[(i)].solution_value):
            print(rutas[i],y[(i)].solution_value)


(1, 0, 0, 1) 0.5
(0, 1, 1, 0) 0.5
(1, 1, 0, 1) 0.5
(1, 0, 1, 1) 0.5


docplex.mp.solution.SolveSolution(obj=9600,values={x_0_1_0_1:50,x_0_2_0_..

In [42]:
matriz=[(i,j)for i in range(numNodos) for j in range(numNodos) ]
vector=[(i) for i in range(numNodos)]
r={}
for i in matriz:
    r[i]=0
    



mdlAux= Model("Aux")
rutaNueva=mdlAux.binary_var_dict(matriz,name="r")
aux=mdlAux.binary_var_dict(vector,name="s")
dualCapacidad = mdl.dual_values(capacidad)


dualMaxRutas=mdl._dual_value1(maxRutas)
objAux=mdlAux.maximize(dualMaxRutas+mdlAux.sum((-rutaNueva[i,j])*dualCapacidad[numNodos*i+j] for i in range(numNodos) for j in range(numNodos)))


mdlAux.add_constraint(mdlAux.sum(rutaNueva[(0,j)] for j in range(1,numNodos))<=aux[(0)])

for i in range(1,numNodos):    
    mdlAux.add_constraint(mdlAux.sum(rutaNueva[(i,j)] for j in range(i+1,numNodos))<=mdlAux.sum(rutaNueva[(j,i)] for j in range(i-1))+aux[(i)])
    
for i in range(numNodos):
    for j in range(i+1,numNodos):
        mdlAux.add_constraint(rutaNueva[(i,j)]==rutaNueva[(j,i)])
        
for i in range(numNodos):
    mdlAux.add_constraint(rutaNueva[(i,i)]==0)

mdlAux.add_constraint(mdlAux.sum(aux[(i)] for i in range(numNodos) )<=1)
                       
mdlAux.solve(log_output=True)
mdlAux.report()

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)

Root node processing (before b&c):
  Real time             =    0.00 sec. (0.00 ticks)
Parallel b&c, 24 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.00 sec. (0.00 ticks)
* model Aux solved with objective = 0.000


In [43]:


nuevaRuta=
for i in range(numNodos):
    for j in range (numNodos):
        print(rutaNueva[(i,j)].solution_value,end=" ")
    print(" ")

print(z)  


 

0 0 0 0  
0 0 0 0  
0 0 0 0  
0 0 0 0  
0


In [45]:
#parametros del problema
b=[]
aux=[]
#parametros
nodos=[1,2,3,4]
rutas=[(1,1,1,1),(1,0,0,1)]
numNodos=len(nodos)
numRutas=len(rutas)

#contruir b
for i in range(numNodos):
     aux.append(0) 
for i in range(numNodos):
     b.append(aux[:])  
aux=1        
for i in range(numNodos):
    for j in range(numNodos):
        if(i != j):
            b[i][j]=50
            aux=aux+1
c=[]
aux=[]
for i in range(numNodos):
     aux.append(0) 
for i in range(numNodos):
     c.append(aux[:])  
for i in range(numNodos):
    for j in range(numNodos):
        if(i != j):
            c[i][j]=abs(i-j)*10-abs(i-j)+1 
            
B2={(o,d,i):0 for o in range(numNodos) for d in range(numNodos) for i in  range(numNodos)}
for i in range(numNodos):
    for j in range(numNodos):
        B2[i,j,i]=b[i][j]
        B2[i,j,j]=-b[i][j]


arcos1=[(k,i,j)for k in range(numRutas) for i in range(numNodos) for j in range(numNodos) ]
rutasmodificadas=[]
rutaaux=[]
for i in range(numRutas):
   rutaaux.clear() 
   for j in range(numNodos):
        if rutas[i][j] == 1:
            rutaaux.append(j)          
   rutasmodificadas.append(rutaaux[:])  

#creacion parametro rkij


r={}
for i in arcos1:
    r[i]=0
for i in range(numRutas):
    for j in range(len(rutasmodificadas[i])-1):
        r[(i,rutasmodificadas[i][j],rutasmodificadas[i][j+1])]=1
        r[(i,rutasmodificadas[i][j+1],rutasmodificadas[i][j])]=1
#-------------------------
        
arcosX=[(i,j,o,d) for i in range(numNodos)for j in range(numNodos)for o in range(numNodos)for d in range(numNodos)]
arcosY=[(i) for i in range (numRutas)]
u= 100
m= 40
Q= 50
alpha=600


In [46]:

from docplex.mp.model import Model


mdl= Model("BRT")
#creacion variables de desicion
x=mdl.continuous_var_dict(arcosX,name="x")
y=mdl.continuous_var_dict(arcosY,name="y")


 # funcion a minimizar
costoX=mdl.sum(c[i][j]*x[(i,j,o,d)]for i in range(numNodos) for j in range(numNodos)for o in range(numNodos)for d in range(numNodos))   
costoY=mdl.sum(alpha*(y[i])for i in range(numRutas))
obj=mdl.minimize(costoX+costoY)



#capacidad  
capacidad=[]
capacidadAux=[]
for i in  range(numNodos):
    for j in range(numNodos): 
        capacidad.append(mdl.add_constraint(mdl.sum(x[(i,j,o,d)] for o in range(numNodos)for d in range(numNodos))<= mdl.sum(r[(k,i,j)]*y[(k)]*u for k in range(numRutas))))
    
#flujo
for i in range(numNodos):  
    for o in range(numNodos):
        for d in range(numNodos):       
            mdl.add_constraint((mdl.sum(x[(i,j,o,d) ] for j in range(numNodos) )-mdl.sum(x[(j,i,o,d) ] for j in range(numNodos) ) )==B2[(o,d,i)])
#maximo rutas
 
sumaRutas= mdl.sum(y[(k)]for k in range(numRutas))
maxRutas=mdl.add_constraint(sumaRutas<=m)  


mdl.solve(log_output=True)
mdl.report()


Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 8 rows and 128 columns.
Reduced LP has 73 rows, 130 columns, and 394 nonzeros.
Presolve time = 0.00 sec. (0.18 ticks)
Symmetry aggregator did 118 additional substitutions.

Iteration log . . .
Iteration:     1   Dual objective     =          1000.000000

Dual crossover.
  Dual:  Fixing 12 variables.
       11 DMoves:  Infeasibility  0.00000000e+00  Objective  1.10000000e+04
        0 DMoves:  Infeasibility  0.00000000e+00  Objective  1.10000000e+04
  Dual:  Pushed 0, exchanged 12.
  Primal:  Fixed no variables.
* model BRT solved with objective = 11000.000


In [47]:
matriz=[(i,j)for i in range(numNodos) for j in range(numNodos) ]
vector=[(i) for i in range(numNodos)]
r={}
for i in matriz:
    r[i]=0
    



mdlAux= Model("Aux")
rutaNueva=mdlAux.binary_var_dict(matriz,name="r")
aux=mdlAux.binary_var_dict(vector,name="s")
dualCapacidad = mdl.dual_values(capacidad)



objAux=mdlAux.maximize(mdlAux.sum((-rutaNueva[i,j])*dualCapacidad[numNodos*i+j] for i in range(numNodos) for j in range(numNodos)))


mdlAux.add_constraint(mdlAux.sum(rutaNueva[(0,j)] for j in range(1,numNodos))<=aux[(0)])

for i in range(1,numNodos):    
    mdlAux.add_constraint(mdlAux.sum(rutaNueva[(i,j)] for j in range(i+1,numNodos))<=mdlAux.sum(rutaNueva[(j,i)] for j in range(i-1))+aux[(i)])
    
for i in range(numNodos):
    for j in range(i+1,numNodos):
        mdlAux.add_constraint(rutaNueva[(i,j)]==rutaNueva[(j,i)])
        
for i in range(numNodos):
    mdlAux.add_constraint(rutaNueva[(i,i)]==0)

mdlAux.add_constraint(mdlAux.sum(aux[(i)] for i in range(numNodos) )<=1)
                       
mdlAux.solve(log_output=True)
mdlAux.report()

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 8 rows and 12 columns.
Aggregator did 7 substitutions.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.02 ticks)

Root node processing (before b&c):
  Real time             =    0.00 sec. (0.03 ticks)
Parallel b&c, 24 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.00 sec. (0.03 ticks)
* model Aux solved with objective = 8.000


In [48]:
for i in range(numNodos):
    for j in range (numNodos):
        print(rutaNueva[(i,j)].solution_value,end=" ")
    print(" ")
z=mdl._dual_value1(maxRutas)
print(z)

0 0 1.0 0  
0 0 0 0  
1.0 0 0 0  
0 0 0 0  
0


In [49]:
#parametros del problema
b=[]
aux=[]
#parametros
nodos=[1,2,3,4]
rutas=[(1,1,1,1),(1,0,0,1),(1,0,1,0)]
numNodos=len(nodos)
numRutas=len(rutas)

#contruir b
for i in range(numNodos):
     aux.append(0) 
for i in range(numNodos):
     b.append(aux[:])  
aux=1        
for i in range(numNodos):
    for j in range(numNodos):
        if(i != j):
            b[i][j]=50
            aux=aux+1
c=[]
aux=[]
for i in range(numNodos):
     aux.append(0) 
for i in range(numNodos):
     c.append(aux[:])  
for i in range(numNodos):
    for j in range(numNodos):
        if(i != j):
            c[i][j]=abs(i-j)*10-abs(i-j)+1 
            
B2={(o,d,i):0 for o in range(numNodos) for d in range(numNodos) for i in  range(numNodos)}
for i in range(numNodos):
    for j in range(numNodos):
        B2[i,j,i]=b[i][j]
        B2[i,j,j]=-b[i][j]


arcos1=[(k,i,j)for k in range(numRutas) for i in range(numNodos) for j in range(numNodos) ]
rutasmodificadas=[]
rutaaux=[]
for i in range(numRutas):
   rutaaux.clear() 
   for j in range(numNodos):
        if rutas[i][j] == 1:
            rutaaux.append(j)          
   rutasmodificadas.append(rutaaux[:])  

#creacion parametro rkij


r={}
for i in arcos1:
    r[i]=0
for i in range(numRutas):
    for j in range(len(rutasmodificadas[i])-1):
        r[(i,rutasmodificadas[i][j],rutasmodificadas[i][j+1])]=1
        r[(i,rutasmodificadas[i][j+1],rutasmodificadas[i][j])]=1
#-------------------------
        
arcosX=[(i,j,o,d) for i in range(numNodos)for j in range(numNodos)for o in range(numNodos)for d in range(numNodos)]
arcosY=[(i) for i in range (numRutas)]
u= 100
m= 40
Q= 50
alpha=600

In [50]:

from docplex.mp.model import Model


mdl= Model("BRT")
#creacion variables de desicion
x=mdl.continuous_var_dict(arcosX,name="x")
y=mdl.continuous_var_dict(arcosY,name="y")


 # funcion a minimizar
costoX=mdl.sum(c[i][j]*x[(i,j,o,d)]for i in range(numNodos) for j in range(numNodos)for o in range(numNodos)for d in range(numNodos))   
costoY=mdl.sum(alpha*(y[i])for i in range(numRutas))
obj=mdl.minimize(costoX+costoY)



#capacidad  
capacidad=[]
capacidadAux=[]
for i in  range(numNodos):
    for j in range(numNodos): 
        capacidad.append(mdl.add_constraint(mdl.sum(x[(i,j,o,d)] for o in range(numNodos)for d in range(numNodos))<= mdl.sum(r[(k,i,j)]*y[(k)]*u for k in range(numRutas))))
    
#flujo
for i in range(numNodos):  
    for o in range(numNodos):
        for d in range(numNodos):       
            mdl.add_constraint((mdl.sum(x[(i,j,o,d) ] for j in range(numNodos) )-mdl.sum(x[(j,i,o,d) ] for j in range(numNodos) ) )==B2[(o,d,i)])
#maximo rutas
 
sumaRutas= mdl.sum(y[(k)]for k in range(numRutas))
maxRutas=mdl.add_constraint(sumaRutas<=m)  


mdl.solve(log_output=True)
mdl.report()


Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 6 rows and 96 columns.
Reduced LP has 75 rows, 163 columns, and 493 nonzeros.
Presolve time = 0.00 sec. (0.20 ticks)
Symmetry aggregator did 42 additional substitutions.

Iteration log . . .
Iteration:     1   Dual objective     =           500.000000

Dual crossover.
  Dual:  Fixing 11 variables.
       10 DMoves:  Infeasibility  0.00000000e+00  Objective  1.09000000e+04
        0 DMoves:  Infeasibility  0.00000000e+00  Objective  1.09000000e+04
  Dual:  Pushed 2, exchanged 9.
  Primal:  Fixed no variables.
* model BRT solved with objective = 10900.000


In [51]:
matriz=[(i,j)for i in range(numNodos) for j in range(numNodos) ]
vector=[(i) for i in range(numNodos)]
r={}
for i in matriz:
    r[i]=0
    



mdlAux= Model("Aux")
rutaNueva=mdlAux.binary_var_dict(matriz,name="r")
aux=mdlAux.binary_var_dict(vector,name="s")
dualCapacidad = mdl.dual_values(capacidad)



objAux=mdlAux.maximize(mdlAux.sum((-rutaNueva[i,j])*dualCapacidad[numNodos*i+j] for i in range(numNodos) for j in range(numNodos)))


mdlAux.add_constraint(mdlAux.sum(rutaNueva[(0,j)] for j in range(1,numNodos))<=aux[(0)])

for i in range(1,numNodos):    
    mdlAux.add_constraint(mdlAux.sum(rutaNueva[(i,j)] for j in range(i+1,numNodos))<=mdlAux.sum(rutaNueva[(j,i)] for j in range(i-1))+aux[(i)])
    
for i in range(numNodos):
    for j in range(i+1,numNodos):
        mdlAux.add_constraint(rutaNueva[(i,j)]==rutaNueva[(j,i)])
        
for i in range(numNodos):
    mdlAux.add_constraint(rutaNueva[(i,i)]==0)

mdlAux.add_constraint(mdlAux.sum(aux[(i)] for i in range(numNodos) )<=1)
                       
mdlAux.solve(log_output=True)
mdlAux.report()

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 8 rows and 12 columns.
Aggregator did 7 substitutions.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.02 ticks)

Root node processing (before b&c):
  Real time             =    0.00 sec. (0.03 ticks)
Parallel b&c, 24 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.00 sec. (0.03 ticks)
* model Aux solved with objective = 8.000


In [52]:
for i in range(numNodos):
    for j in range (numNodos):
        print(rutaNueva[(i,j)].solution_value,end=" ")
    print(" ")
z=mdl._dual_value1(maxRutas)
print(z)

0 0 0 0  
0 0 0 1.0  
0 0 0 0  
0 1.0 0 0  
0


In [58]:
#parametros del problema
b=[]
aux=[]
#parametros
nodos=[1,2,3,4]
rutas=[(1,1,1,1),(1,1,1,0),(1,1,0,1),(1,0,1,1),(0,1,1,1),(0,0,1,1),(0,1,0,1),(0,1,1,0),(1,0,0,1),
      (1,0,1,0),(1,1,0,0)]
numNodos=len(nodos)
numRutas=len(rutas)

#contruir b
for i in range(numNodos):
     aux.append(0) 
for i in range(numNodos):
     b.append(aux[:])  
aux=1        
for i in range(numNodos):
    for j in range(numNodos):
        if(i != j):
            b[i][j]=50
            aux=aux+1
c=[]
aux=[]
for i in range(numNodos):
     aux.append(0) 
for i in range(numNodos):
     c.append(aux[:])  
for i in range(numNodos):
    for j in range(numNodos):
        if(i != j):
            c[i][j]=abs(i-j)*10-abs(i-j)+1 
            
B2={(o,d,i):0 for o in range(numNodos) for d in range(numNodos) for i in  range(numNodos)}
for i in range(numNodos):
    for j in range(numNodos):
        B2[i,j,i]=b[i][j]
        B2[i,j,j]=-b[i][j]


arcos1=[(k,i,j)for k in range(numRutas) for i in range(numNodos) for j in range(numNodos) ]
rutasmodificadas=[]
rutaaux=[]
for i in range(numRutas):
   rutaaux.clear() 
   for j in range(numNodos):
        if rutas[i][j] == 1:
            rutaaux.append(j)          
   rutasmodificadas.append(rutaaux[:])  

#creacion parametro rkij


r={}
for i in arcos1:
    r[i]=0
for i in range(numRutas):
    for j in range(len(rutasmodificadas[i])-1):
        r[(i,rutasmodificadas[i][j],rutasmodificadas[i][j+1])]=1
        r[(i,rutasmodificadas[i][j+1],rutasmodificadas[i][j])]=1
#-------------------------
        
arcosX=[(i,j,o,d) for i in range(numNodos)for j in range(numNodos)for o in range(numNodos)for d in range(numNodos)]
arcosY=[(i) for i in range (numRutas)]
u= 100
m= 2
alpha=600

In [59]:

from docplex.mp.model import Model


mdl= Model("BRT")
#creacion variables de desicion
x=mdl.continuous_var_dict(arcosX,name="x")
y=mdl.continuous_var_dict(arcosY,name="y")


 # funcion a minimizar
costoX=mdl.sum(c[i][j]*x[(i,j,o,d)]for i in range(numNodos) for j in range(numNodos)for o in range(numNodos)for d in range(numNodos))   
costoY=mdl.sum(alpha*(y[i])for i in range(numRutas))
obj=mdl.minimize(costoX+costoY)



#capacidad  
capacidad=[]
capacidadAux=[]
for i in  range(numNodos):
    for j in range(numNodos): 
        capacidad.append(mdl.add_constraint(mdl.sum(x[(i,j,o,d)] for o in range(numNodos)for d in range(numNodos))<= mdl.sum(r[(k,i,j)]*y[(k)]*u for k in range(numRutas))))
    
#flujo
for i in range(numNodos):  
    for o in range(numNodos):
        for d in range(numNodos):       
            mdl.add_constraint((mdl.sum(x[(i,j,o,d) ] for j in range(numNodos) )-mdl.sum(x[(j,i,o,d) ] for j in range(numNodos) ) )==B2[(o,d,i)])
#maximo rutas
 
sumaRutas= mdl.sum(y[(k)]for k in range(numRutas))
maxRutas=mdl.add_constraint(sumaRutas<=m)  


mdl.solve(log_output=True)
mdl.report()

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 4 rows and 64 columns.
Reduced LP has 77 rows, 203 columns, and 621 nonzeros.
Presolve time = 0.00 sec. (0.22 ticks)
Symmetry aggregator did 162 additional substitutions.

Iteration log . . .
Iteration:     1   Dual objective     =          1000.000000

Dual crossover.
  Dual:  Fixing 18 variables.
       17 DMoves:  Infeasibility  0.00000000e+00  Objective  1.08000000e+04
        0 DMoves:  Infeasibility  0.00000000e+00  Objective  1.08000000e+04
  Dual:  Pushed 1, exchanged 17.
  Primal:  Fixed no variables.
* model BRT solved with objective = 10800.000


In [60]:
matriz=[(i,j)for i in range(numNodos) for j in range(numNodos) ]
vector=[(i) for i in range(numNodos)]
r={}
for i in matriz:
    r[i]=0
    



mdlAux= Model("Aux")
rutaNueva=mdlAux.binary_var_dict(matriz,name="r")
aux=mdlAux.binary_var_dict(vector,name="s")
dualCapacidad = mdl.dual_values(capacidad)



objAux=mdlAux.maximize(mdlAux.sum((-rutaNueva[i,j])*dualCapacidad[numNodos*i+j] for i in range(numNodos) for j in range(numNodos)))


mdlAux.add_constraint(mdlAux.sum(rutaNueva[(0,j)] for j in range(1,numNodos))<=aux[(0)])

for i in range(1,numNodos):    
    mdlAux.add_constraint(mdlAux.sum(rutaNueva[(i,j)] for j in range(i+1,numNodos))<=mdlAux.sum(rutaNueva[(j,i)] for j in range(i-1))+aux[(i)])
    
for i in range(numNodos):
    for j in range(i+1,numNodos):
        mdlAux.add_constraint(rutaNueva[(i,j)]==rutaNueva[(j,i)])
        
for i in range(numNodos):
    mdlAux.add_constraint(rutaNueva[(i,i)]==0)

mdlAux.add_constraint(mdlAux.sum(aux[(i)] for i in range(numNodos) )<=1)
                       
mdlAux.solve(log_output=True)
mdlAux.report()

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 8 rows and 12 columns.
Aggregator did 7 substitutions.
All rows and columns eliminated.
Presolve time = 0.02 sec. (0.02 ticks)

Root node processing (before b&c):
  Real time             =    0.02 sec. (0.03 ticks)
Parallel b&c, 24 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.02 sec. (0.03 ticks)
* model Aux solved with objective = 6.000


In [61]:
for i in range(numNodos):
    for j in range (numNodos):
        print(rutaNueva[(i,j)].solution_value,end=" ")
    print(" ")
z=mdl._dual_value1(maxRutas)
print(z)

0 0 1.0 0  
0 0 0 0  
1.0 0 0 0  
0 0 0 0  
0


In [62]:
dir(mdl)

['DEFAULT_OBJECTIVE_FMT',
 'DEFAULT_VAR_VALUE_QUOTED_SOLUTION_FMT',
 'DEFAULT_VAR_VALUE_UNQUOTED_SOLUTION_FMT',
 '_Model__allpwlfuncs',
 '_Model__engine',
 '_Model__notify_new_model_object',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_batch_logical_cts',
 '_add_constraint_internal',
 '_add_indicator',
 '_add_progress_listener',
 '_add_pwl_constraint_internal',
 '_add_pwl_expr',
 '_add_sos',
 '_add_var_container',
 '_aggregator',
 '_all_containers',
 '_allkpis',
 '_apply_parameters_to_engine',
 '_benders_annotations',
 '_binary_vartype',
 '_can_solve',
 '_change_var_types_internal',
 '_check_basis',
 '_check_has_multi_ob